In [1]:
import json
import os
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.utils import Progbar
from tensorflow.keras.metrics import Mean
from pqdm.threads import pqdm
from tqdm import tqdm

2024-08-11 13:28:31.438649: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 13:28:31.438672: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 13:28:31.439454: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-11 13:28:31.443457: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-11 13:28:32.060288: W tensorflow/compiler/tf2

In [2]:
from transformers import AutoTokenizer, TFT5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
# model = TFT5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

# tokenizer = AutoTokenizer.from_pretrained("models/v1")
model = TFT5ForConditionalGeneration.from_pretrained("models/v1")

2024-08-11 13:28:33.756715: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-11 13:28:33.758040: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-11 13:28:33.779515: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
def load_jsonl(path):
    with open(path) as input_file:
        lines = list(map(json.loads, input_file))

    return lines

def load_data(root_path):
    files = os.listdir(root_path)
    dataset= {}

    for filename in files:
        filepath = root_path + f'/{filename}'

        dataset[filename] = load_jsonl(filepath)

    return dataset

In [4]:
dataset = load_data('data/L1')

In [5]:
models = ['llama3.1', 'gemma2:2b', 'gemma2']

test_from = 'attrebute_val'
train_from = 'attrebute_train'
target_from = 'attrebute_test'

train_inputs = train_from + '.data'
train_labels = train_from + '.solution'

test_inputs = test_from + '.data'
test_labels = test_from + '.solution'

target_inputs = target_from + '.data'

test_inputs = dataset[test_inputs]
test_labels = dataset[test_labels]
train_inputs = dataset[train_inputs]
train_labels = dataset[train_labels]
target_inputs = dataset[target_inputs]

In [6]:
SEQ_LEN=250

def make_prompt(data):
    if 'title' in data:
        input_type = 'input'
    else:
        input_type = 'label'

    prompt = {
        "label": "The product belongs to these categories:\nBrand: {details_Brand}\nLevel 0: {L0_category}\nLevel 1: {L1_category}\nLevel 2: {L2_category}\nLevel 3: {L3_category}\nLevel 4: {L4_category}",
        "input": "Categories the product into 5 levels of categories and identify the brand:\nTitle: {title}\nStore:{store}\nManufacturer:{details_Manufacturer}"
    }

    return prompt[input_type].format(**data)

def batch_make_prompt(data):
    return [make_prompt(item) for item in data]
# def tokenize()

In [7]:
train_input_prompts = pqdm(train_inputs, make_prompt, n_jobs=5)
train_label_prompts = pqdm(train_labels, make_prompt, n_jobs=5)
test_input_prompts = pqdm(test_inputs, make_prompt, n_jobs=5)
test_label_prompts = pqdm(test_labels, make_prompt, n_jobs=5)

QUEUEING TASKS | :   0%|          | 0/443499 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/443499 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/443499 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/443499 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/443499 [00:00<?, ?it/s]

...

PROCESSING TASKS | :   0%|          | 0/95035 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/95035 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/95035 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/95035 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/95035 [00:00<?, ?it/s]

In [9]:
def train_model(inputs, labels):
    inputs = [item.decode() for item in inputs]
    labels = [item.decode() for item in labels]

    encoding = tokenizer(
        inputs,
        padding="max_length",
        max_length=SEQ_LEN,
        truncation=True,
        return_tensors="np",
    )

    input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

    target_encoding = tokenizer(
        labels,
        padding="max_length",
        max_length=SEQ_LEN,
        truncation=True,
        return_tensors="np",
    )

    labels = target_encoding.input_ids
    labels[labels == tokenizer.pad_token_id] = -100

    return model(input_ids=input_ids, attention_mask=attention_mask, labels=labels).loss

def predict(inputs):
    encoding = tokenizer.batch_encode_plus(
        inputs,
        padding="max_length",
        max_length=SEQ_LEN,
        truncation=True,
        return_tensors="np",
    )

    input_ids, attention_mask = encoding.input_ids, encoding.attention_mask
    outputs = model.generate(input_ids, max_new_tokens=SEQ_LEN)

    return outputs

In [8]:
BATCH_SIZE = 64
train_ds = tf.data.Dataset.from_tensor_slices((train_input_prompts, train_label_prompts)).shuffle(1000).batch(BATCH_SIZE)
test_ds = tf.data.Dataset.from_tensor_slices((test_input_prompts, test_label_prompts)).shuffle(1000).batch(BATCH_SIZE)

In [ ]:
optimizer = keras.optimizers.AdamW(learning_rate=0.001)

In [ ]:
EPOCHS=5

per_epoch_loss=[]

for e in range(EPOCHS):
    dataset_iter = train_ds.as_numpy_iterator()

    progbar = Progbar(len(train_ds), stateful_metrics=['train_loss'])
    loss_metric = Mean("train_loss")

    for i, batch in enumerate(dataset_iter):
        inputs, labels = batch

        with tf.GradientTape() as tape:
            loss = train_model(inputs, labels)

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        loss_metric.update_state(loss)
        values = [('train_loss', loss_metric.result())]

        progbar.update(i+1, values=values)

In [ ]:
model.save_pretrained('./models/v1')

In [ ]:
dataset_iter = test_ds.as_numpy_iterator()

progbar = Progbar(len(test_ds), stateful_metrics=['val_loss'])
loss_metric = Mean("val_loss")

for i, batch in enumerate(dataset_iter):
    inputs, labels = batch

    loss = train_model(inputs, labels)

    loss_metric.update_state(loss)
    values = [('val_loss', loss_metric.result())]

    progbar.update(i+1, values=values)

In [ ]:
len(test_ds), len(train_ds)

In [10]:
target_prompts = pqdm(target_inputs, make_prompt, n_jobs=5)

QUEUEING TASKS | :   0%|          | 0/95036 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/95036 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/95036 [00:00<?, ?it/s]

In [82]:
def tokenizer_data(item):
    item = item.numpy().decode('utf-8')
    encoding = tokenizer.encode(
        item,
        padding="max_length",
        max_length=SEQ_LEN,
        truncation=True,
        return_tensors="np",
    )
    return encoding[0]

def batch_tokenize(batch):
    encoding = tokenizer.batch_encode_plus(
        batch,
        padding="max_length",
        max_length=SEQ_LEN,
        truncation=True,
        return_tensors="np",
    )
    return encoding.input_ids

In [60]:
target_ds = (tf.data.Dataset.from_tensor_slices(target_prompts)
             .map(lambda a: tf.py_function(tokenizer_data, [a], [tf.int8]))
             .batch(128))

In [68]:
def chunk_list(lst, n):
    # Split the list into chunks of n items
    chunks = [lst[i:i + n] for i in range(0, len(lst), n)]
    return chunks

In [98]:
batches = chunk_list(target_prompts, n=1024)
batches = pqdm(batches, batch_tokenize, n_jobs=12)

QUEUEING TASKS | :   0%|          | 0/93 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/93 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/93 [00:00<?, ?it/s]

In [99]:
answers = []

for input_ids in tqdm(batches):
    outputs = model.generate(input_ids, max_new_tokens=SEQ_LEN)

    answers.extend(tokenizer.batch_decode(outputs))

100%|██████████| 93/93 [26:51<00:00, 17.32s/it]


In [180]:
import regex as re
word=r'.'
pattern = rf"(Brand:\s?{word}+)? (Level 0:\s?{word}+)? (Level 1:\s?{word}+)? (Level 2:\s?{word}+)? (Level 3:\s?{word}+)? (Level 4:\s?{word}+)?"

def extract_categories(output):

    brand, l0, l1, l2, l3, l4 = re.findall(pattern, output)[0]
    return {
        'details_Brand': brand,
        'L0_category': l0,
        'L1_category': l1,
        'L2_category': l2,
        'L3_category': l3,
        'L4_category': l4
    }
print(pattern)


(Brand:\s?.+)? (Level 0:\s?.+)? (Level 1:\s?.+)? (Level 2:\s?.+)? (Level 3:\s?.+)? (Level 4:\s?.+)?


In [201]:
import re

def extract_categories(text):
    key_map = {
        'Brand': 'details_Brand',
        'Level 0': 'L0_category',
        'Level 1': 'L1_category',
        'Level 2': 'L2_category',
        'Level 3': 'L3_category',
        'Level 4': 'L4_category'
    }
    
    # Regex patterns to match each key
    patterns = {
        'Brand': r'Brand:\s*(.+?)(?=\s+Level|\s*<\/s>)',
        'Level 0': r'Level 0:\s*(.+?)(?=\s+Level|\s*<\/s>)',
        'Level 1': r'Level 1:\s*(.+?)(?=\s+Level|\s*<\/s>)',
        'Level 2': r'Level 2:\s*(.+?)(?=\s+Level|\s*<\/s>)',
        'Level 3': r'Level 3:\s*(.+?)(?=\s+Level|\s*<\/s>)',
        'Level 4': r'Level 4:\s*(.+?)(?=\s+Level|\s*<\/s>)'
    }
    
    # Extracting data using the patterns
    extracted_data = {v: 'na' for v in key_map.values()}
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            extracted_data[key_map[key]] = match.group(1).strip()
    
    return extracted_data

# Example usage
text = """
The product belongs to these categories: Brand: LUXPaper Level 0: Office Products Level 1: Office & School Supplies Level 2: Envelopes, Mailers & Shipping Supplies Level 3: Envelopes Level 4: Greeting Card Envelopes</s><pad><pad><pad><pad><pad><pad><pad><pad>
"""

result = extract_categories(text)
print(result)


{'details_Brand': 'LUXPaper', 'L0_category': 'Office Products', 'L1_category': 'Office & School Supplies', 'L2_category': 'Envelopes, Mailers & Shipping Supplies', 'L3_category': 'Envelopes', 'L4_category': 'Greeting Card Envelopes'}


In [202]:
final_outputs = []
for i, item in enumerate(tqdm(answers)):
    details = {'indoml_id': i}
    details.update(extract_categories(item))

    final_outputs.append(json.dumps(details) + '\n')

100%|██████████| 95036/95036 [00:01<00:00, 89530.47it/s]


In [204]:
output_path = './data/L1/output/'

with open(output_path + 'attribute_test_topjourney_1.predict', 'w') as output_file:
    output_file.writelines(final_outputs)

In [198]:
item = answers[0]
matches = re.findall(pattern, item)[0]
key_map = {
    'Brand':'details_Brand',
    'Level 0':'L0_category',
    'Level 1':'L1_category',
    'Level 2':'L2_category',
    'Level 3':'L3_category',
    'Level 4':'L4_category' 
}

details = dict.fromkeys(key_map.values(), '')
for match in matches:
    key, value = list(map(str.strip, item.split(':')))
    key = key_map[key]

    details[key] = value



ValueError: too many values to unpack (expected 2)

In [200]:
item

'<pad> The product belongs to these categories: Brand: CURT Level 0: Automotive Level 1: Exterior Accessories Level 2: Towing Products & Winches Level 3: Hitch Accessories Level 4: Wiring</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [199]:
matches

('Brand: CURT',
 'Level 0: Automotive',
 'Level 1: Exterior Accessories',
 'Level 2: Towing Products & Winches',
 'Level 3: Hitch Accessories Level 4:',
 '')

In [191]:
item

''

In [189]:
key, value = list(map(str.strip, matches[0].split(':')))
key = key_map[key]

['Brand', 'Breeze']

In [171]:
extract_categories(item)

ValueError: too many values to unpack (expected 6)